# Palety *Flat UI Colors*

```
```

Struktura:
- każda paleta składa się z **20** kolorów zorganizowanych w siatce **4 x 5**,
- rodzicem siatki jest `div` o klasie `colors`,
- każdy kolor reprezentowany jest przez `div` o klasie `color`,
- nazwa koloru to `span` będący bezpośrednim dzieckiem bloku, nie posiada żadnej charakterystycznej klasy;

Formatem docelowym jest zbiór natywnych zmiennych CSS lub zbiór zmiennych języka Less, opcjonalnie zorganizowanych w mapie klucz-wartość. Kluczem/nazwą zmiennej będzie nazwa koloru nadana przez projektanta, w notacji kebab case. 

Niektóre nazwy mają znaki niedozwolone w zmiennej – program pozostawia je w takiej samej formie, pozwalając na dostosowanie ich we własnym zakresie.

Style parsowane są za pomocą wyrażeń regularnych, ponieważ moduł `cssutils` nie posiada jeszcze udokumentowanej zdolności parsowania pojedynczych zasad zamiast całych arkuszy.

In [1]:
import re
from time import sleep

from selenium import webdriver
from bs4 import BeautifulSoup

```

```
## Deklaracje

In [2]:
def extract_color(element):
    name = element.find('span').string
    style = element['style']
    
    color = re.search(r'background(?:\-color)*:\s*(rgb(?:a)*.*)\;', style).group(1)
    color = parse_rgb(color)
    
    return (name, color)

In [3]:
def get_palette(driver, abbr: str) -> dict:
    url = 'https://flatuicolors.com/palette/{}'.format(abbr)
    site = driver.get(url)
    sleep(1)
    soup = BeautifulSoup(driver.page_source)
    driver.close()
    
    color_nodes = soup.find_all('div', class_='color')
    extracted = [extract_color(node) for node in color_nodes]
    colors = {name: rgb_to_hex(color) for name, color in extracted}
    
    return colors

```

```

In [4]:
def kebab(name: str):
    return name.lower().replace(' ', '-')

In [5]:
def format_var(name: str, color: str, prefix=''):
    template = '{}: {};'
    name = kebab(name)
    
    return prefix + template.format(name, color)

In [6]:
def format_ruleset(container: str, colors: dict, prefix: str):
    template = '{}: {{\n    {}\n}}';
    colors = [format_var(name, color, prefix) for name, color in colors.items()]
    ruleset = '\n    '.join(colors)
    
    return template.format(container, ruleset)

In [7]:
def format_less_ruleset(name: str, colors: dict):
    return format_ruleset('@' + name, colors, '')

def format_css_ruleset(colors: dict, container: str = ':root'):
    return format_ruleset(container, colors, '--')

In [8]:
def format_less_variables(colors: dict):
    variables = [format_var(name, color, '@') for name, color in colors.items()]
    
    return '\n'.join(variables)

```

```

In [9]:
def rgb_to_hex(rgb):
    rgb = tuple(rgb)
    return '#%02x%02x%02x' % rgb

In [10]:
def parse_rgb(rgb):
    pattern = r'rgb\(\s*(\d{1,3})\s*,\s*(\d{1,3})\s*,\s*(\d{1,3})\s*\)'
    match = re.search(pattern, rgb)
    distinct = [int(value) for value in match.group(1, 2, 3)]
    
    return distinct

```

```
## Użycie

In [11]:
driver = webdriver.Chrome('chromedriver')

In [12]:
prefix = 'es'
colors = get_palette(driver, prefix)

In [13]:
print(format_less_variables(colors))

@jacksons-purple: #40407a;
@c64-purple: #706fd3;
@swan-white: #f7f1e3;
@summer-sky: #34ace0;
@celestial-green: #33d9b2;
@lucky-point: #2c2c54;
@liberty: #474787;
@hot-stone: #aaa69d;
@devil-blue: #227093;
@palm-springs-splash: #218c74;
@fluorescent-red: #ff5252;
@synthetic-pumpkin: #ff793f;
@crocodile-tooth: #d1ccc0;
@mandarin-sorbet: #ffb142;
@spiced-butternut: #ffda79;
@eye-of-newt: #b33939;
@chilean-fire: #cd6133;
@grey-porcelain: #84817a;
@alameda-ochre: #cc8e35;
@desert: #ccae62;


In [14]:
print(format_less_ruleset(prefix, colors))

@es: {
    jacksons-purple: #40407a;
    c64-purple: #706fd3;
    swan-white: #f7f1e3;
    summer-sky: #34ace0;
    celestial-green: #33d9b2;
    lucky-point: #2c2c54;
    liberty: #474787;
    hot-stone: #aaa69d;
    devil-blue: #227093;
    palm-springs-splash: #218c74;
    fluorescent-red: #ff5252;
    synthetic-pumpkin: #ff793f;
    crocodile-tooth: #d1ccc0;
    mandarin-sorbet: #ffb142;
    spiced-butternut: #ffda79;
    eye-of-newt: #b33939;
    chilean-fire: #cd6133;
    grey-porcelain: #84817a;
    alameda-ochre: #cc8e35;
    desert: #ccae62;
}


In [15]:
print(format_css_ruleset(colors))

:root: {
    --jacksons-purple: #40407a;
    --c64-purple: #706fd3;
    --swan-white: #f7f1e3;
    --summer-sky: #34ace0;
    --celestial-green: #33d9b2;
    --lucky-point: #2c2c54;
    --liberty: #474787;
    --hot-stone: #aaa69d;
    --devil-blue: #227093;
    --palm-springs-splash: #218c74;
    --fluorescent-red: #ff5252;
    --synthetic-pumpkin: #ff793f;
    --crocodile-tooth: #d1ccc0;
    --mandarin-sorbet: #ffb142;
    --spiced-butternut: #ffda79;
    --eye-of-newt: #b33939;
    --chilean-fire: #cd6133;
    --grey-porcelain: #84817a;
    --alameda-ochre: #cc8e35;
    --desert: #ccae62;
}
